In [25]:
from flask import Flask, request, jsonify
import pandas as pd
import pickle

In [26]:
df = pd.read_csv('../csv/cleaned_data_car.csv')

In [29]:
model_and_columns = pickle.load(open('../models/decision_tree_with_columns.sav', 'rb'))
model = model_and_columns['model']
columns = model_and_columns['columns']

In [34]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if request.headers['Content-Type'] == 'application/json':
            data = request.get_json(force=True)

            if 'features' in data and isinstance(data['features'], dict):
                features_dict = data['features']
                
                non_zero_features = [features_dict.get(col, 0) for col in columns]
                
                features_df = pd.DataFrame([non_zero_features], columns=columns)
                
                prediction = model.predict(features_df)
                
                prediction = int(prediction[0])
                
                return jsonify({'prediction': prediction})
            else:
                return jsonify({'error': 'Invalid or missing "features" data in the request'}), 400
        else:
            return jsonify({'error': 'Invalid content type. Use application/json'}), 415
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return jsonify({'error': f'An internal server error occurred: {str(e)}'}), 500



if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2023 17:14:20] "POST /predict HTTP/1.1" 200 -
